In [10]:
import os,re,subprocess
from typing import Optional
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
from openai import OpenAI
from dotenv import load_dotenv
from templates import *
import xml.etree.ElementTree as ET
import json
import subprocess
from pathlib import Path
load_dotenv()
from e2e.encrypt_local import *
from cryptography.fernet import Fernet
import json
from constants import * 


In [12]:

class AWSCLI:
    def __init__(self, region_name="us-east-1"):
        """
        Initialize the AWSChecker class with a specific region.
        """
        self.region_name = region_name
        try:
            self.ec2_client = boto3.client("ec2", region_name=self.region_name)
        except (NoCredentialsError, PartialCredentialsError) as e:
            print(f"Error with AWS credentials: {e}")
            raise
    
    def list_keypairs(self):
        """
        List all key pairs in the region.
        """
        try:
            response = self.ec2_client.describe_key_pairs()
            key_pairs = response.get("KeyPairs", [])
            return [{"KeyName": kp["KeyName"], "KeyFingerprint": kp["KeyFingerprint"]} for kp in key_pairs]
        except self.ec2_client.exceptions.ClientError as e:
            print(f"Error fetching key pairs: {e}")
            return []

    
    def check_region(self):
        """
        Verify the current region and list available regions.
        """
        ec2 = boto3.client("ec2")
        response = ec2.describe_regions()
        available_regions = [region["RegionName"] for region in response["Regions"]]
        is_region_valid = self.region_name in available_regions
        return {
            "current_region": self.region_name,
            "is_valid": is_region_valid,
            "available_regions": available_regions
        }
    
    def check_vpc(self):
        """
        List all VPCs in the region.
        """
        response = self.ec2_client.describe_vpcs()
        vpcs = response.get("Vpcs", [])
        return [{"VpcId": vpc["VpcId"], "CidrBlock": vpc["CidrBlock"]} for vpc in vpcs]
    
    def check_subnets(self):
        """
        List all subnets in the region.
        """
        response = self.ec2_client.describe_subnets()
        subnets = response.get("Subnets", [])
        return [{"SubnetId": subnet["SubnetId"], "VpcId": subnet["VpcId"], "CidrBlock": subnet["CidrBlock"]} for subnet in subnets]
    
    def check_security_groups(self):
        """
        List all security groups in the region.
        """
        response = self.ec2_client.describe_security_groups()
        security_groups = response.get("SecurityGroups", [])
        return [{"GroupId": sg["GroupId"], "GroupName": sg["GroupName"], "Description": sg["Description"]} for sg in security_groups]

    def list_running_instances(self):
        """
        List all running EC2 instances with their instance ID, state, and IP addresses.
        """
        try:
            response = self.ec2_client.describe_instances(
                Filters=[{"Name": "instance-state-name", "Values": ["running"]}]
            )
            instances_info = []
            for reservation in response.get("Reservations", []):
                for instance in reservation.get("Instances", []):
                    instance_info = {
                        "InstanceId": instance.get("InstanceId"),
                        "InstanceType": instance.get("InstanceType"),
                        "PrivateIpAddress": instance.get("PrivateIpAddress"),
                        "PublicIpAddress": instance.get("PublicIpAddress"),
                        "State": instance.get("State", {}).get("Name"),
                        "Tags": instance.get("Tags", [])
                    }
                    instances_info.append(instance_info)
            return instances_info
        except self.ec2_client.exceptions.ClientError as e:
            print(f"Error fetching instances: {e}")
            return []

In [13]:
x = AWSCLI()



In [14]:
x.list_running_instances()

[{'InstanceId': 'i-08e11f4664ac337ab',
  'InstanceType': 't3.large',
  'PrivateIpAddress': '172.31.30.197',
  'PublicIpAddress': '54.221.6.27',
  'State': 'running',
  'Tags': [{'Key': 'Name', 'Value': 'Jenkins-auto-anchor-v32'}]}]